In [ ]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-
############################
#File Name: recommendator.py
#Author: yanbin
#Mail: yanbin918@gmail.com
#Created Time: 2017-08-17 10:
from ml_latest_small.data import *
from Collaborative_Filter.itemBased import ItemCFModel,ItemCF
from Collaborative_Filter.userBased import UserCFModel,UserCF
from Collaborative_Filter.lfmBased import lfmModel,lfm
from Comparision.randomBased import RandomModel
from Comparision.topnBased import topnModel
import pickle
import os
from Evaluation.Evaluator import Evaluator

def load_result(path):
    pkl_file = open(path, 'rb')
    prediction_dict = pickle.load(pkl_file)
    # pprint.pprint(data)
    pkl_file.close()
    return prediction_dict

def evaluate(evaluator):
    precise = evaluator.precision()
    coverage = evaluator.coverage()
    popularity = evaluator.popularity()
    recall = evaluator.recall()
    return precise,recall,coverage,popularity

def itemCF_predict(train_dataset,test_dataset,K):
    train_dict = train_dataset.user_item_dict
    test_dict = test_dataset.user_item_dict
    itemCF = ItemCF()
    #triain ItemCF_model
    print "训练基于物品协同过滤算法模型........"
    ItemCF_model=itemCF.train(train_dict)
    #recommendation for every user in train_Dataset
    users = train_dict.keys()
    user_nums = len(users)
    i = 0
    prediction_dict ={}
    for user in users:
        prediction_dict[user]=ItemCF_model.recommendation(user,config={'K':K,'N':10}).keys()
    #将每位用户预测结果存入本地
    output = open(result_pt+itemCF_prediction_result_pt %K, 'wb')
    # Pickle dictionary using protocol 0.
    pickle.dump(prediction_dict, output)
    print "物品协同过滤推荐算法预测结果存入本地!"
    # Pickle the list using the highest protocol available.
    output.close()
    return prediction_dict

def userCF_predict(train_dataset,test_dataset,K):
    train_dict = train_dataset.user_item_dict
    test_dict = test_dataset.user_item_dict    
    userCF = UserCF()
    #triain ItemCF_model
    print "用户基于物品协同过滤算法模型........"
    UserCF_model=userCF.train(train_dict)
    #recommendation for every user in train_Dataset
    users = train_dict.keys()
    user_nums = len(users)
    i = 0
    prediction_dict ={}
    for user in users:
        prediction_dict[user]=UserCF_model.recommendation(user,config={'K':K,'N':10}).keys()
    #将每位用户预测结果存入本地
    output = open(result_pt+userCF_prediction_result_pt %K, 'wb')
    # Pickle dictionary using protocol 0.bb
    pickle.dump(prediction_dict, output)
    print "用户协同过滤推荐算法预测结果存入本地!"
    # Pickle the list using the highest protocol available.
    output.close()
    return prediction_dict

def random_predict(train_dataset,test_dataset,K):
    train_dict = train_dataset.user_item_dict
    test_dict = test_dataset.user_item_dict
    randomRM = RandomModel(train_dict)
    #triain ItemCF_model
    print "基于随机推荐算法模型........"
    #recommendation for every user in train_Dataset
    users = train_dict.keys()
    user_nums = len(users)
    i = 0
    prediction_dict ={}
    for user in users:
        prediction_dict[user]=randomRM.recommendation(user,config={'K':K,'N':10}).keys()
    #将每位用户预测结果存入本地
    output = open(result_pt+random_prediction_result_pt %K, 'wb')
    # Pickle dictionary using protocol 0.
    pickle.dump(prediction_dict, output)
    print "基于随机推荐算法预测结果存入本地!"
    # Pickle the list using the highest protocol available.
    output.close()
    return prediction_dict
    
def topn_predict(train_dataset,test_dataset):
    train_dict = train_dataset.user_item_dict
    test_dict = test_dataset.user_item_dict
    topn_model = topnModel()
    topn_model.fit(train_dict)
    #triain ItemCF_model
    print "热门商品推荐算法模型........"
    #recommendation for every user in train_Dataset
    users = train_dict.keys()
    user_nums = len(users)
    i = 0
    prediction_dict ={}
    for user in users:
        prediction_dict[user]=topn_model.recommendation(user,config={'N':10}).keys()
    #将每位用户预测结果存入本地
    output = open(result_pt+topn_prediction_result_pt, 'wb')
    # Pickle dictionary using protocol 0.
    pickle.dump(prediction_dict, output)
    print "热门商品推荐算法预测结果存入本地!"
    # Pickle the list using the highest protocol available.
    output.close()
    return prediction_dict

def lfm_predict(train_dataset,test_dataset,F=10,lmbd=0.01,alpha=0.9):
    train_dict = train_dataset.user_item_dict
    test_dict = test_dataset.user_item_dict
    LFM = lfm(F=F,lmbd=lmbd,alpha=alpha,max_iter = 100)
    lfm_model= LFM.fit(train_dict)
    #triain lfm_model
    print "LFM推荐算法模型........"
    #recommendation for every user in train_Dataset
    users = train_dict.keys()
    user_nums = len(users)
    i = 0
    prediction_dict ={}
    for user in users:
        prediction_dict[user]=lfm_model.recommendation(user,config={'N':10}).keys()
    #将每位用户预测结果存入本地
    output = open(result_pt+lfm_prediction_result_pt %(F,lmbd,alpha), 'wb')
    # Pickle dictionary using protocol 0.
    pickle.dump(prediction_dict, output)
    print "LFM推荐算法预测结果存入本地!"
    # Pickle the list using the highest protocol available.
    output.close()
    return prediction_dict

inputPath = "./ml_latest_small/ratings.csv"
inputPath = '/home/yanbin/data/ml-100k/u.data'
result_pt = "./Predict_Result/"
userCF_prediction_result_pt ='UserCF_Result/%s_usercf_prediction_dict.pkl'
itemCF_prediction_result_pt ='ItemCF_Result/%s_itemcf_prediction_dict.pkl'
random_prediction_result_pt = 'Random_Result/%s_random_prediction_dict.pkl'
topn_prediction_result_pt = 'Topn_Result/topn_prediction_dict.pkl'
lfm_prediction_result_pt = 'LFM_Result/%s_%s_%s_lfm_prediction_dict.pkl'
recommendation_enginer = {0:random_prediction_result_pt,
                          1:topn_prediction_result_pt,
                          2:userCF_prediction_result_pt,
                          3:itemCF_prediction_result_pt,
                          4:lfm_prediction_result_pt}
recommendator = {0:'randomRM',1:'topnRM',2:'userCF',3:'itemCF',4:'lfm'}
train_dataset,test_dataset = read_data_sets(inputPath,with_split=True,split_char='\t')
Ks = [5,10,20,40,80,160]
i = input("please choose recommendation engineer:\n0->randomRM, 1->topnRM, 2->userBased, 3->itemBased, 4->lfm\n")
assert i  in [0,1,2,3,4] ,"recomendation enginer not exists"
print "The recommendator is "+recommendator[i]
if i in [0,2,3]:
    for K in Ks:
        path = (result_pt+recommendation_enginer[i] %K)
        flag = os.path.exists(path)
        if flag:prediction_dict = load_result(path)
        elif i is 0:
                prediction_dict = random_predict(train_dataset,test_dataset,K)
        elif i is 2:
            prediction_dict = userCF_predict(train_dataset,test_dataset,K)
        elif i is 3:
            prediction_dict = itemCF_predict(train_dataset,test_dataset,K)
        evaluator = Evaluator(train_dataset.user_item_dict,test_dataset.user_item_dict,prediction_dict,N=10)
        (precise,recall,coverage,popularity) = evaluate(evaluator)
        print ('K:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
               %(K,precise*100,recall*100,coverage*100,popularity))

elif i is 1:
    path = (result_pt+recommendation_enginer[i])
    flag = os.path.exists(path)
    prediction_dict = load_result(path) if flag else\
    topn_predict(train_dataset,test_dataset)
    evaluator = Evaluator(train_dataset.user_item_dict,test_dataset.user_item_dict,prediction_dict,N=10)
    (precise,recall,coverage,popularity) = evaluate(evaluator)
    print ('precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
           %(precise*100,recall*100,coverage*100,popularity))
elif i is 4:
    Fs = [10,20]
    lmbds = [0.001,0.003,0.009,0.018,0.054,0.108]
    alphas = [0.001,0.003,0.009,0.018,0.054,0.108]
    for F in Fs:
        for lmbd in lmbds:
            for alpha in alphas:
                path = (result_pt+recommendation_enginer[i] % (F,lmbd,alpha) )
                flag = os.path.exists(path)
                prediction_dict = load_result(path) if flag else\
                lfm_predict(train_dataset,test_dataset,F=F,lmbd=lmbd,alpha=alpha)
                evaluator = Evaluator(train_dataset.user_item_dict,test_dataset.user_item_dict,prediction_dict,N=10)
                (precise,recall,coverage,popularity) = evaluate(evaluator)
                print ('F:%3d, lmbd:%.4f, alpha:%4f, precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
                       %(F,lmbd,alpha,precise*100,recall*100,coverage*100,popularity))

please choose recommendation engineer:
0->randomRM, 1->topnRM, 2->userBased, 3->itemBased, 4->lfm
4
The recommendator is lfm
F: 10, lmbd:0.0010,alpha:0.001000, precise:7.85%,  recall:4.42%,  coverage:7.74%,  popularity:4.69
F: 10, lmbd:0.0010,alpha:0.003000, precise:1.69%,  recall:0.95%,  coverage:10.70%,  popularity:3.63
F: 10, lmbd:0.0010,alpha:0.009000, precise:1.36%,  recall:0.77%,  coverage:14.63%,  popularity:3.43
F: 10, lmbd:0.0010,alpha:0.018000, precise:3.13%,  recall:1.76%,  coverage:37.12%,  popularity:3.49
F: 10, lmbd:0.0010,alpha:0.054000, precise:4.18%,  recall:2.36%,  coverage:51.09%,  popularity:3.85


## RandomRM
随机推荐商品，模型评价指标情况如下

In [5]:
for K in Ks:
    path = (result_pt+random_prediction_result_pt %K)
    flag = os.path.exists(path)
    prediction_dict = load_result(path) if flag else \
    random_predict(train_dataset,test_dataset,K)
    evaluator = Evaluator(train_dataset.user_item_dict,
                          test_dataset.user_item_dict,
                          prediction_dict,N=10)
    (precise,recall,coverage,popularity) = evaluate(evaluator)
    print ('K:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
           %(K/5,precise*100,recall*100,coverage*100,popularity))

K:  1,  precise:0.28%,  recall:0.11%,  coverage:0.12%,  popularity:1.43
K:  2,  precise:0.22%,  recall:0.09%,  coverage:53.84%,  popularity:1.64
K:  4,  precise:0.30%,  recall:0.12%,  coverage:54.78%,  popularity:1.65
K:  8,  precise:0.21%,  recall:0.08%,  coverage:54.09%,  popularity:1.64
K: 16,  precise:0.12%,  recall:0.05%,  coverage:54.90%,  popularity:1.63
K: 32,  precise:0.28%,  recall:0.11%,  coverage:54.60%,  popularity:1.63


## TopnRM
推荐每个用户，流行度最高N件商品 

In [24]:
path = (result_pt+topn_prediction_result_pt)
flag = os.path.exists(path)
prediction_dict = load_result(path) if flag else \
topn_predict(train_dataset,test_dataset)
evaluator = Evaluator(train_dataset.user_item_dict,
                      test_dataset.user_item_dict,
                      prediction_dict,N=10)
(precise,recall,coverage,popularity) = evaluate(evaluator)
print ('N:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
       %(10,precise*100,recall*100,coverage*100,popularity))

N: 10,  precise:6.96%,  recall:2.79%,  coverage:0.12%,  popularity:5.46


# UserCF
### 基于用户的协同过滤算法
待补充

In [25]:
for K in Ks:
    path = (result_pt+userCF_prediction_result_pt %K)
    flag = os.path.exists(path)
    prediction_dict = load_result(path) if flag else \
    userCF_predict(train_dataset,test_dataset,K)
    evaluator = Evaluator(train_dataset.user_item_dict,
                          test_dataset.user_item_dict,
                          prediction_dict,N=10)
    (precise,recall,coverage,popularity) = evaluate(evaluator)
    print ('K:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
           %(K,precise*100,recall*100,coverage*100,popularity))

K:  5,  precise:17.70%,  recall:7.10%,  coverage:9.90%,  popularity:4.59
K: 10,  precise:19.97%,  recall:8.01%,  coverage:6.87%,  popularity:4.77
K: 20,  precise:21.88%,  recall:8.78%,  coverage:4.81%,  popularity:4.92
K: 40,  precise:22.88%,  recall:9.18%,  coverage:3.18%,  popularity:5.04
K: 80,  precise:22.18%,  recall:8.89%,  coverage:2.25%,  popularity:5.14
K:160,  precise:21.13%,  recall:8.48%,  coverage:1.64%,  popularity:5.23


# ItemCF
### 基于物品的协同过滤算法
待补充

In [26]:
for K in Ks:
    path = (result_pt+itemCF_prediction_result_pt %K)
    flag = os.path.exists(path)
    prediction_dict = load_result(path) if flag else \
    itemCF_predict(train_dataset,test_dataset,K)
    evaluator = Evaluator(train_dataset.user_item_dict,
                          test_dataset.user_item_dict,
                          prediction_dict,N=10)
    (precise,recall,coverage,popularity) = evaluate(evaluator)
    print ('K:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
           %(K,precise*100,recall*100,coverage*100,popularity))

K:  5,  precise:20.39%,  recall:8.18%,  coverage:8.79%,  popularity:4.58
K: 10,  precise:21.91%,  recall:8.79%,  coverage:6.54%,  popularity:4.78
K: 20,  precise:21.83%,  recall:8.76%,  coverage:5.46%,  popularity:4.84
K: 40,  precise:22.24%,  recall:8.92%,  coverage:4.92%,  popularity:4.87
K: 80,  precise:22.47%,  recall:9.01%,  coverage:4.64%,  popularity:4.88
K:160,  precise:22.00%,  recall:8.82%,  coverage:4.81%,  popularity:4.83


# LFM
###  基于隐语义模型的推荐
待补充

In [29]:
Fs = [10,100]
for F in Fs:
    path = (result_pt+lfm_prediction_result_pt %F)
    flag = os.path.exists(path)
    prediction_dict = load_result(path) if flag else\
    lfm_predict(train_dataset,test_dataset,F=F)
    evaluator = Evaluator(train_dataset.user_item_dict,
                          test_dataset.user_item_dict,
                          prediction_dict,N=10)
    (precise,recall,coverage,popularity) = evaluate(evaluator)
    print ('F:%3d,  precise:%2.2f%%,  recall:%2.2f%%,  coverage:%2.2f%%,  popularity:%2.2f'
           %(F,precise*100,recall*100,coverage*100,popularity))

F: 10,  precise:0.46%,  recall:0.19%,  coverage:2.35%,  popularity:1.82


KeyboardInterrupt: 